In [ ]:
from random import gauss
from random import seed
from pandas import Series
import numpy as np
from matplotlib import pyplot as plt
from sklearn.linear_model import Ridge
import sklearn
import functionList

In [ ]:
###### THIS IS THE CODE BLOCK THAT CREATES THE SIMULATED DATA ################


l = 5000 #change the length of the data if you want
eSeriesX = np.zeros((5,l))
c = 5

for i in range(5):
    seed(i)
    series = [gauss(0.0, 1.0) for i in range(l)]
    eSeriesX[i,:] = Series(series)

X = np.zeros((5,l))
for i in range(l):
    X[0,i] = .95*np.sqrt(2)*X[0,i-1] - .9025*X[0,i-2] + eSeriesX[0,i]
    X[1,i] = .5*X[0,i-2] + eSeriesX[1,i]
    X[2,i] = -.4*X[0,i-3] + eSeriesX[2,i]
    X[3,i] = -.5*X[0,i-2] + .25*np.sqrt(2)*X[3,i-1] + .25*np.sqrt(2)*X[4,i-1] + eSeriesX[3,i]
    X[4,i] = -.25*np.sqrt(2)*X[3,i-1] + .25*np.sqrt(2)*X[4,i-1] + eSeriesX[4,i]
X = np.transpose(X)


In [ ]:
memVec = [1, 2, 3, 4, 5]
ridgeVec = [.1, .5, 1]
use_NormY_Regularizer = 1 #include a switch between normY and length of Y as a regularizer

cvNum = 3

if use_NormY_Regularizer == 1:
    s = (len(memVec), len(ridgeVec), c, cvNum)
    error = np.zeros(s)
    for memNumber in memVec:
        featureMat = functionList.buildFeatureMat(X,memNumber)
        xClean = X
        for regularizer in ridgeVec: 
            crossVal = sklearn.model_selection.KFold(n_splits=cvNum)
            for thisChannel in range(c):
                adjustedReg = regularizer*np.linalg.norm(xClean[:,thisChannel])
                cvRun = 0
                for train, test in crossVal.split(featureMat):
                    pred = functionList.predRrOutput(featureMat[train,:], featureMat[test,:], xClean[train,thisChannel], adjustedReg, 0)
                    err = functionList.findMSE(pred, xClean[test,thisChannel])
                    error[memVec.index(memNumber), ridgeVec.index(regularizer), thisChannel, cvRun] = err
                    cvRun = cvRun + 1
    errNoCv = np.mean(error, 3)
    errNoCh = np.mean(errNoCv, 2)
else:
    s = (len(memVec), len(ridgeVec), cvNum)
    error = np.zeros(s)
    ridgeVecNormal = np.dot(ridgeVec,l) #use a regularization parameter that is based on length
    for memNumber in memVec:
        featureMat = functionList.buildFeatureMat(X,memNumber)
        xClean = X
        for regularizer in ridgeVecNormal: 
            RR = Ridge(alpha=regularizer)
            crossVal = sklearn.model_selection.KFold(n_splits=cvNum)
            cvRun = 0
            for train, test in crossVal.split(featureMat):
                RR.fit(featureMat[train,:],xClean[train])
                pred = RR.predict(featureMat[test])
                error[memVec.index(memNumber), ridgeVec.index(regularizer/l), cvRun] = functionList.findMSE(pred, xClean[test])
                cvRun = cvRun + 1
    errNoCh = np.mean(error, 2)

In [ ]:
showChannels = 1 #toggel between 0 and 1 for channel by channel info
#this is only valid if you used use_NormY_Regularizer == 1

if showChannels == 1 & use_NormY_Regularizer == 1:
    for ch in range(c):
        titleString = f"Memory vs MSE, channel {ch}"
        plt.title(titleString)
        for i in range(len(ridgeVec)):
            plt.plot(memVec[:], errNoCv[: , i, ch], ".", label = f"lambda = {ridgeVec[i]}") ## figure out columns to pass
        plt.legend()
        plt.show()

if use_NormY_Regularizer == 1:
    titleString = f"Memory vs MSE, channel's averaged"
else:
    titleString = f"Memory vs MSE"
plt.title(titleString)
for i in range(len(ridgeVec)):
    plt.plot(memVec[:], errNoCh[:, i], ".", label = f"lambda = {ridgeVec[i]}") ## figure out columns to pass
    plt.xlabel("Number of memory points")
    plt.ylabel("MSE")
plt.legend()
plt.show()